<a href="https://colab.research.google.com/github/Shellyga/Adversarial-Domain-Adaptation-with-Keras/blob/master/Shelly_Adversarial_Domain_Adaptation_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow
print(tensorflow.__version__)

1.15.2


# Driver

In [0]:
SEED = 7
import os
import sys
import argparse
import random
import numpy as np
# import tensorflow.python.keras as tf
from tensorflow.compat.v1 import set_random_seed
# import tensorflow.python.keras as tf

os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
set_random_seed(SEED)
random.seed(SEED)

from PIL import Image
from keras.utils import to_categorical
from keras.layers import Input
from keras.optimizers import Adam
from keras.utils import multi_gpu_model
from sklearn.metrics import accuracy_score
# import model
# import optimizer

def pil_loader(path):
    # print(path)
    # Return the RGB variant of input image
    with open(path, 'rb') as f:
      with Image.open(f) as img:
        return img.convert('RGB')

def one_hot_encoding(param):
    # Read the source and target labels from param
    s_label = param["source_label"]
    t_label = param["target_label"]

    # Encode the labels into one-hot format
    classes = (np.concatenate((s_label, t_label), axis = 0))
    num_classes = np.max(classes)
    if 0 in classes:
            num_classes = num_classes+1
    s_label = to_categorical(s_label, num_classes = num_classes)
    t_label = to_categorical(t_label, num_classes = num_classes)
    return s_label, t_label
            
def data_loader(filepath, inp_dims):
    # Load images and corresponding labels from the text file, stack them in numpy arrays and return
    if not os.path.isfile(filepath):
        print("File path {} does not exist. Exiting...".format(filepath))
        # sys.exit() 
    img = []
    label = []
    with open(filepath,'r',encoding='utf-8-sig') as fp:
        for line in fp:
            token = line.split()
            # print('drive/My Drive/project_data/'+token[0])
            image_path = "drive/My Drive/project_data/"+token[0]
            i = pil_loader(image_path)
            i = i.resize((inp_dims[0], inp_dims[1]), Image.ANTIALIAS)
            img.append(np.array(i))
            label.append(int(token[1]))
    img = np.array(img)
    label = np.array(label)
    return img, label

def batch_generator(data, batch_size):
    #Generate batches of data.
    all_examples_indices = len(data[0])
    while True:
        mini_batch_indices = np.random.choice(all_examples_indices, size = batch_size, replace = False)
        tbr = [k[mini_batch_indices] for k in data]
        yield tbr

def train(param):
    models = {}
    inp = Input(shape = (param["inp_dims"]))
    embedding = build_embedding(param, inp)
    classifier = build_classifier(param, embedding)
    # classifier = build_classifier(param)
    discriminator = build_discriminator(param, embedding)

    if param["number_of_gpus"] > 1:
        models["combined_classifier"] = multi_gpu_model(build_combined_classifier(inp, classifier), gpus = param["number_of_gpus"])
        models["combined_discriminator"] = multi_gpu_model(build_combined_discriminator(inp, discriminator), gpus = param["number_of_gpus"])
        models["combined_model"] = multi_gpu_model(build_combined_model(inp, [classifier, discriminator]), gpus = param["number_of_gpus"])
    else:
        models["combined_classifier"] = build_combined_classifier(inp, classifier)
        models["combined_discriminator"] = build_combined_discriminator(inp, discriminator)
        models["combined_model"] = build_combined_model(inp, [classifier, discriminator])

    models["combined_classifier"].compile(optimizer = opt_classifier(param), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    models["combined_discriminator"].compile(optimizer = opt_discriminator(param), loss = 'binary_crossentropy', metrics = ['accuracy'])
    models["combined_model"].compile(optimizer = opt_combined(param), loss = {'class_act_last': 'categorical_crossentropy', 'dis_act_last': \
        'binary_crossentropy'}, loss_weights = {'class_act_last': param["class_loss_weight"], 'dis_act_last': param["dis_loss_weight"]}, metrics = ['accuracy'])
     
    Xs, ys = param["source_data"], param["source_label"]
    Xt, yt = param["target_data"], param["target_label"]

    # Xs_train, Xs_test, Ys_train, Ys_test = train_test_split(Xs, ys, test_size=0.25, random_state=0)
    # Xt_train, Xt_test, Yt_train, Yt_test = train_test_split(Xt, yt, test_size=0.25, random_state=0)

    # Source domain is represented by label 0 and Target by 1
    ys_adv = np.array(([0.] * ys.shape[0]))
    yt_adv = np.array(([1.] * yt.shape[0]))

    y_advb_1 = np.array(([1] * param["batch_size"] + [0] * param["batch_size"])) # For gradient reversal
    y_advb_2 = np.array(([0] * param["batch_size"] + [1] * param["batch_size"]))
    weight_class = np.array(([1] * param["batch_size"] + [0] * param["batch_size"]))
    weight_adv = np.ones((param["batch_size"] * 2,))
    S_batches = batch_generator([Xs, ys], param["batch_size"])
    T_batches = batch_generator([Xt, np.zeros(shape = (len(Xt),))], param["batch_size"])

    param["target_accuracy"] = 0

    optim = {}
    optim["iter"] = 0
    optim["acc"] = ""
    optim["labels"] = np.array(Xt.shape[0],)
    gap_last_snap = 0

    for i in range(param["num_iterations"]):        
        Xsb, ysb = next(S_batches)
        Xtb, ytb = next(T_batches)
        X_adv = np.concatenate([Xsb, Xtb])
        y_class = np.concatenate([ysb, np.zeros_like(ysb)])

        adv_weights = []
        for layer in models["combined_model"].layers:
            if (layer.name.startswith("dis_")):
                adv_weights.append(layer.get_weights())
          
        stats1 = models["combined_model"].train_on_batch(X_adv, [y_class, y_advb_1],\
                                sample_weight=[weight_class, weight_adv])            
        k = 0
        for layer in models["combined_model"].layers:
            if (layer.name.startswith("dis_")):                    
                layer.set_weights(adv_weights[k])
                k += 1

        class_weights = []        
        for layer in models["combined_model"].layers:
            if (not layer.name.startswith("dis_")):
                class_weights.append(layer.get_weights())  

        stats2 = models["combined_discriminator"].train_on_batch(X_adv, [y_advb_2])

        k = 0
        for layer in models["combined_model"].layers:
            if (not layer.name.startswith("dis_")):
                layer.set_weights(class_weights[k])
                k += 1

        if ((i + 1) % param["test_interval"] == 0):
            ys_pred = models["combined_classifier"].predict(Xs)
            yt_pred = models["combined_classifier"].predict(Xt)
            ys_adv_pred = models["combined_discriminator"].predict(Xs)
            yt_adv_pred = models["combined_discriminator"].predict(Xt)

            source_accuracy = accuracy_score(ys.argmax(1), ys_pred.argmax(1))              
            target_accuracy = accuracy_score(yt.argmax(1), yt_pred.argmax(1))
            source_domain_accuracy = accuracy_score(ys_adv, np.round(ys_adv_pred))              
            target_domain_accuracy = accuracy_score(yt_adv, np.round(yt_adv_pred))
            log_str = "iter: {:05d}: \nLABEL CLASSIFICATION: source_accuracy: {:.5f}, target_accuracy: {:.5f}\
                    \nDOMAIN DISCRIMINATION: source_domain_accuracy: {:.5f}, target_domain_accuracy: {:.5f} \n"\
                                                         .format(i, source_accuracy*100, target_accuracy*100,
                                                      source_domain_accuracy*100, target_domain_accuracy*100)
            print(log_str)

            if param["target_accuracy"] < target_accuracy:              
                optim["iter"] = i
                optim["acc"] = log_str
                # optim["labels"] = ys_pred.argmax(1)

                if (gap_last_snap >= param["snapshot_interval"]):
                    gap_last_snap = 0
                    np.save(os.path.join(param["output_path"],"yPred_{}".format(optim["iter"])), optim["labels"])
                    open(os.path.join(param["output_path"], "acc_{}.txt".format(optim["iter"])), "w").write(optim["acc"])
                    models["combined_classifier"].save(os.path.join(param["output_path"],"iter_{:05d}_model.h5".format(i)))
        gap_last_snap = gap_last_snap + 1;

if __name__ == "__main__":
    # Read parameter values from the console
    parser = argparse.ArgumentParser(description = 'Domain Adaptation')
    parser.add_argument('--number_of_gpus', type = int, nargs = '?', default = '1', help = "Number of gpus to run")
    parser.add_argument('--network_name', type = str, default = 'ResNet50', help = "Name of the feature extractor network")
    parser.add_argument('--dataset_name', type = str, default = 'Office', help = "Name of the source dataset")
    parser.add_argument('--dropout_classifier', type = float, default = 0.25, help = "Dropout ratio for classifier")
    parser.add_argument('--dropout_discriminator', type = float, default = 0.25, help = "Dropout ratio for discriminator")    
    parser.add_argument('--source_path', type = str, default = 'amazon_10_list.txt', help = "Path to source dataset")
    parser.add_argument('--target_path', type = str, default = 'webcam_10_list.txt', help = "Path to target dataset")
    parser.add_argument('--lr_classifier', type = float, default = 0.0001, help = "Learning rate for classifier model")
    parser.add_argument('--b1_classifier', type = float, default = 0.9, help = "Exponential decay rate of first moment \
                                                                                             for classifier model optimizer")
    parser.add_argument('--b2_classifier', type = float, default = 0.999, help = "Exponential decay rate of second moment \
                                                                                            for classifier model optimizer")
    parser.add_argument('--lr_discriminator', type = float, default = 0.00001, help = "Learning rate for discriminator model")
    parser.add_argument('--b1_discriminator', type = float, default = 0.9, help = "Exponential decay rate of first moment \
                                                                                             for discriminator model optimizer")
    parser.add_argument('--b2_discriminator', type = float, default = 0.999, help = "Exponential decay rate of second moment \
                                                                                            for discriminator model optimizer")
    parser.add_argument('--lr_combined', type = float, default = 0.00001, help = "Learning rate for combined model")
    parser.add_argument('--b1_combined', type = float, default = 0.9, help = "Exponential decay rate of first moment \
                                                                                             for combined model optimizer")
    parser.add_argument('--b2_combined', type = float, default = 0.999, help = "Exponential decay rate of second moment \
                                                                                            for combined model optimizer")
    parser.add_argument('--classifier_loss_weight', type = float, default = 1, help = "Classifier loss weight")
    parser.add_argument('--discriminator_loss_weight', type = float, default = 4, help = "Discriminator loss weight")
    parser.add_argument('--batch_size', type = int, default = 32, help = "Batch size for training")
    parser.add_argument('--test_interval', type = int, default = 3, help = "Gap between two successive test phases")
    parser.add_argument('--num_iterations', type = int, default = 12000, help = "Number of iterations")
    parser.add_argument('--snapshot_interval', type = int, default = 500, help = "Minimum gap between saving outputs")
    parser.add_argument('--output_dir', type = str, default = 'Models', help = "Directory for saving outputs")
    # args = parser.parse_args()

    # Set GPU device
    # os.environ["CUDA_VISIBLE_DEVICES"] = str(list(np.arange(args.number_of_gpus))).strip('[]')

    # Initialize parameters
    param = {}
    param["number_of_gpus"] = 1
    param["network_name"] = 'ResNet50'
    param["inp_dims"] = [224, 224, 3]
    # param["num_iterations"] = 12000
    param["num_iterations"] = 500
    param["lr_classifier"] = 0.0001
    param["b1_classifier"] = 0.9
    param["b2_classifier"] = 0.999    
    param["lr_discriminator"] = 0.00001
    param["b1_discriminator"] =  0.9
    param["b2_discriminator"] = 0.999
    param["lr_combined"] = 0.00001
    param["b1_combined"] =  0.9
    param["b2_combined"] =  0.999       
    param["batch_size"] = int(32/2)
    param["class_loss_weight"] = 1
    param["dis_loss_weight"] = 4    
    param["drop_classifier"] = 0.25
    param["drop_discriminator"] = 0.25
    param["test_interval"] = 3
    param["source_path"] = 'drive/My Drive/project_data/your_file.txt'
    param["target_path"] = 'drive/My Drive/project_data/your_file_shelly.txt' 
    # param["snapshot_interval"] = 500
    param["snapshot_interval"] = 5
    param["output_path"] = 'drive/My Drive/project_data/result2'

    # Create directory for saving models and log files
    if not os.path.exists(param["output_path"]):
        os.mkdir(param["output_path"])
    
    # Load source and target data
    param["source_data"], param["source_label"] = data_loader(param["source_path"], param["inp_dims"])
    param["target_data"], param["target_label"] = data_loader(param["target_path"], param["inp_dims"])

    # Encode labels into one-hot format
    param["source_label"], param["target_label"] = one_hot_encoding(param)

    # Train data
    train(param)


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
iter: 00002: 
LABEL CLASSIFICATION: source_accuracy: 42.50000, target_accuracy: 60.83333                    
DOMAIN DISCRIMINATION: source_domain_accuracy: 80.83333, target_domain_accuracy: 40.00000 

iter: 00005: 
LABEL CLASSIFICATION: source_accuracy: 45.00000, target_accuracy: 63.33333                    
DOMAIN DISCRIMINATION: source_domain_accuracy: 93.33333, target_domain_accuracy: 68.33333 

iter: 00008: 
LABEL CLASSIFICATION: source_accuracy: 49.16667, target_accuracy: 65.00000                    
DOMAIN DISCRIMINATION: source_domain_accuracy: 97.50000, target_domain_accuracy: 83.33333 

iter: 00011: 
LABEL CLASSIFICATION: source_accuracy: 55.00000, target_accuracy: 65.00000                    
DOMAIN DISCRIMINATION: source_domain_accuracy: 98.33333, target_domain_accuracy: 88.33333 

iter: 00014: 
LABEL CLASSIFICATION: source_accuracy: 62.50000, target_accuracy: 65.83333                    
DOMAIN DISCRIMINATION:

# Traib

In [0]:
# def train(param):
#     models = {}
#     inp = Input(shape = (param["inp_dims"]))
#     embedding = build_embedding(param, inp)
    
#     discriminator = build_discriminator(param, embedding)
#     models["combined_discriminator"] = build_combined_discriminator(inp, discriminator)     
#     models["combined_discriminator"].compile(optimizer = opt_discriminator(param), loss = 'binary_crossentropy', metrics = ['accuracy'])
   
#     Xs, ys = param["source_data"], param["source_label"]
#     Xt, yt = param["target_data"], param["target_label"]

#     # Source domain is represented by label 0 and Target by 1
#     ys_adv = np.array(([0.] * ys.shape[0]))
#     yt_adv = np.array(([1.] * yt.shape[0]))

#     y_advb_1 = np.array(([1] * param["batch_size"] + [0] * param["batch_size"])) # For gradient reversal
#     y_advb_2 = np.array(([0] * param["batch_size"] + [1] * param["batch_size"]))
#     weight_class = np.array(([1] * param["batch_size"] + [0] * param["batch_size"]))
#     weight_adv = np.ones((param["batch_size"] * 2,))
#     S_batches = batch_generator([Xs, ys], param["batch_size"])
#     T_batches = batch_generator([Xt, np.zeros(shape = (len(Xt),))], param["batch_size"])

#     param["target_accuracy"] = 0

#     optim = {}
#     optim["iter"] = 0
#     optim["acc"] = ""
#     optim["labels"] = np.array(Xt.shape[0],)
#     gap_last_snap = 0

#     for i in range(param["num_iterations"]):        
#         Xsb, ysb = next(S_batches)
#         Xtb, ytb = next(T_batches)
#         X_adv = np.concatenate([Xsb, Xtb])
#         y_class = np.concatenate([ysb, np.zeros_like(ysb)])

#         adv_weights = []  

#         stats2 = models["combined_discriminator"].train_on_batch(X_adv, [y_advb_2])

#         if ((i + 1) % param["test_interval"] == 0):

#             ys_adv_pred = models["combined_discriminator"].predict(Xs)
#             yt_adv_pred = models["combined_discriminator"].predict(Xt)
#             source_domain_accuracy = accuracy_score(ys_adv, np.round(ys_adv_pred))              
#             target_domain_accuracy = accuracy_score(yt_adv, np.round(yt_adv_pred))
#             print(source_domain_accuracy)
#             print(target_domain_accuracy)


In [0]:
with open('drive/My Drive/project_data/your_file_shelly.txt','r',encoding='utf-8-sig') as fp:
        for line in fp:
          token = line.split()
          print(token[0])
          image_path = "drive/My Drive/project_data/"+token[0]
          # print(image_path)
          with open(image_path, 'rb') as f:
              with Image.open(f) as img:
                  image= img.convert('RGB')

# Model

In [0]:
import random
import numpy as np
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense
from keras.layers import BatchNormalization, Activation, Dropout
# from keras_vggface.vggface import VGGFace
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split

def build_embedding(param, inp):
    network = eval(param["network_name"])
    base = network(weights = 'imagenet', include_top = False)
    feat = base(inp)
    flat = Flatten()(feat)
    return flat

def build_classifier(param, embedding):
    dense1 = Dense(400, name = 'class_dense1')(embedding)
    bn1 = BatchNormalization(name = 'class_bn1')(dense1)
    act1 = Activation('relu', name = 'class_act1')(bn1)
    drop2 = Dropout(param["drop_classifier"], name = 'class_drop1')(act1)

    dense2 = Dense(100, name = 'class_dense2')(drop2)
    bn2 = BatchNormalization(name = 'class_bn2')(dense2)
    act2 = Activation('relu', name = 'class_act2')(bn2)
    drop2 = Dropout(param["drop_classifier"], name = 'class_drop2')(act2)

    densel = Dense(param["source_label"].shape[1], name = 'class_dense_last')(drop2)
    bnl = BatchNormalization(name = 'class_bn_last')(densel)
    actl = Activation('softmax', name = 'class_act_last')(bnl)
    return actl

def build_discriminator(param, embedding):
    dense1 = Dense(400, name = 'dis_dense1')(embedding)
    bn1 = BatchNormalization(name='dis_bn1')(dense1)
    act1 = Activation('relu', name = 'dis_act1')(bn1)
    drop1 = Dropout(param["drop_discriminator"], name = 'dis_drop1')(act1)

    dense2 = Dense(100, name = 'dis_dense2')(drop1)
    bn2 = BatchNormalization(name='dis_bn2')(dense2)
    act2 = Activation('relu', name = 'dis_act2')(bn2)
    drop2 = Dropout(param["drop_discriminator"], name = 'dis_drop2')(act2)

    densel = Dense(1, name = 'dis_dense_last')(drop2)
    bnl = BatchNormalization(name = 'dis_bn_last')(densel)
    actl = Activation('sigmoid', name = 'dis_act_last')(bnl)
    return actl

def build_combined_classifier(inp, classifier):
    comb_model = Model(inputs = inp, outputs = [classifier])
    return comb_model

def build_combined_discriminator(inp, discriminator):
    comb_model = Model(inputs = inp, outputs = [discriminator])
    return comb_model

def build_combined_model(inp, comb):
    comb_model = Model(inputs = inp, outputs = comb)
    return comb_model

In [0]:
pip install keras_vggface

# Optimizer

In [2]:
import numpy as np
from keras.optimizers import Adam

def opt_classifier(param):
    return Adam(lr=param["lr_classifier"], beta_1=param["b1_classifier"], beta_2=param["b2_classifier"])

def opt_discriminator(param):
    return Adam(lr=param["lr_discriminator"], beta_1=param["b1_discriminator"], beta_2=param["b2_discriminator"])

def opt_combined(param):
    return Adam(lr=param["lr_combined"], beta_1=param["b1_combined"], beta_2=param["b2_combined"])


Using TensorFlow backend.


# Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
